In [1]:
import requests
import bs4
import json
import html2text
import re
import math
import pandas as pd
from urllib.parse import urlparse

In [3]:
hotentries = [tags.find("link").get_text() for tags in hotentry_soup.find_all("item")]

In [4]:
entry_info_endpoint = "https://b.hatena.ne.jp/entry/jsonlite"

bookmark_users = []
for hotentry in hotentries:
    entry_info = json.loads(requests.get(f"{entry_info_endpoint}/?url={hotentry}").content.decode("utf-8"))
    bookmark_users.extend([bookmark["user"] for bookmark in entry_info["bookmarks"]])

In [6]:
len(set(bookmark_users))

4275

In [1]:
reg = re.compile("\(.*?\)$")
bookmark_per_page = 20
hatena_str = "//blog.hatena.ne.jp"

bookmark_feed_endpoint = "https://b.hatena.ne.jp/{}/bookmark.rss"

schemes = []
domains = []
err_count = 0
for idx, user in enumerate(set(bookmark_users)):
    user_domains = []
    res = requests.get(bookmark_feed_endpoint.format(user))
    soup = bs4.BeautifulSoup(res.content,'lxml-xml').find("description").get_text()
    bookmark_num = int(reg.search(soup).group(0).lstrip("(").rstrip(")").replace(",", ""))

    page_num = math.floor((bookmark_per_page + bookmark_num - 1) / bookmark_per_page)
    for page_idx in range(page_num):
        res = requests.get(f"{bookmark_feed_endpoint.format(user)}?page={page_idx + 1}")
        soup = bs4.BeautifulSoup(res.content,'lxml-xml')
        
        for item in soup.find_all("item"):
            try:
                url = item.find("link").get_text()
                scheme = urlparse(url).scheme
                domain = urlparse(url).netloc
                if domain not in domains:
                    if hatena_str in requests.get(f"{scheme}://{domain}").content.decode("utf-8"):
                        user_domains.append(domain)
                        domains.append(domain)
                        schemes.append(scheme)
            except Exception as e:
                err_count += 1
                continue
        break
         
    if idx % 10 == 0:
        print(f"end_user_num: {idx}, num_domain: {len(domains)}")
   

In [35]:
df = pd.DataFrame({"scheme": schemes, "domain": domains})

In [36]:
df.to_csv("hatena_domain.csv", index=False)

In [49]:
urls = []
for s, d in zip(np.array(schemes)[indecis],  np.array(domains)[indecis]):
    url_str = f"{s}://{d}"
    urls.append(url_str)